<a href="https://colab.research.google.com/github/palkrishna/palkrishna/blob/main/Krishna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Path loss


In [27]:
import numpy as np
from math import log10,sqrt
import scipy.stats
from random import randint
def pathloss(linkType,f_c,d) :
  bsAntGain = 8             # BS antenna gain 8 dBi 
  bsNoiseFigure = 7         # BS noise figure 7 dB 
  d2dAntGain = 5            # vehicle antenna gain 5 dBi
  d2dNoiseFigure = 13       # vehicle noise figure 13 dB
  if ((linkType).upper== 'V2V') :                                                            
    if d<18:
       pathloss= 38.77 + 20 * log10(d) + 20 * log10(f_c)       # LOS pathloss
       PL_dB = -(np.random.randn()*3 + pathloss) + 2*d2dAntGain - d2dNoiseFigure 
    else :
       pathloss = 36.85 + 30 * log10(d) + 18.9 * log10(f_c)    # NLOS pathloss 
       PL_dB = -(np.random.randn()*4 + pathloss) + 2*d2dAntGain - d2dNoiseFigure
  
  else :
      pathloss =  32.4 + 20 * log10(f_c) + 30 * log10(d) 
      PL_dB = -(np.random.randn()*7.8 + pathloss) + d2dAntGain + bsAntGain - bsNoiseFigure 
  
  return PL_dB

User Initialization

In [28]:
def genDUE(numDUE):
  d2dPos = []          # initilizer for vehicle position
  indDUE = []
  indDUE2 = [] 
  pproc2 = []
  numLane = 4
  laneWidth = 4
  v = 50                   # velocity
  d_avg = 2.5*v/3.6         # average inter-vehicle distance according to TR 36.885
  radius = 500
  disBstoHwy = 25           # in meters
  d0 = sqrt(radius**2-disBstoHwy**2)
  ###### generate all d2d positions and store in d2dPos
  for ilane in range(numLane) :
    npoints = np.random.poisson(2*d0/d_avg)
    pproc = (np.random.randn(npoints)*2-1)*100                                      # horizontal coordinates
    for i in range(len(pproc)) :
      z=np.zeros((1,2))
      z[0,0] = pproc[i]
      z[0,1] = disBstoHwy+ilane*laneWidth
      pproc2.append(z)
  d2dPos = pproc2
  numd2d = len(d2dPos)

  indPerm = np.random.permutation(numd2d)
  indDUE = indPerm[0:numDUE]
  indDUE2 = np.zeros(numDUE)
  for ii in range(numDUE)  : 
    ######### pair each element in indDUE with closet vehicle and store the index in indDUE2  ##############
    minDist = 2*d0
    tmpInd = 0
    for iii in range(numd2d) :
        if (abs(iii-indDUE)<1e-6).any() or (abs(iii-indDUE2)<1e-6).any() :                       # iii in indDUE or indDUE2 
            continue
        newDist = sqrt((d2dPos[indDUE[ii]][0][0]-d2dPos[iii][0][0])**2 + (d2dPos[indDUE[ii]][0][1]-d2dPos[iii][0][1])**2)
        if newDist < minDist :
            tmpInd = iii
            minDist = newDist
    indDUE2[ii] = tmpInd                              # the closest DUE pair
  return d2dPos,indDUE,indDUE2

def genCUE(xMin,xMax,yMin,yMax,numCUE) :
  xDelta=xMax-xMin
  yDelta=yMax-yMin
  areaTotal=xDelta*yDelta
  lambda0=100
  numbPoints =scipy.stats.poisson( lambda0*areaTotal ).rvs()#Poisson number of points
  xx = xDelta*scipy.stats.uniform.rvs(0,1,((numbPoints,1)))+xMin#x coordinates of Poisson points
  yy = yDelta*scipy.stats.uniform.rvs(0,1,((numbPoints,1)))+yMin#y coordinates of Poisson points
  x = []
  y= []
  for i in range(numCUE) :
   x.append(xx[i])
   y.append(yy[i])
   return x,y


Main file

In [ ]:
fc = 2                     # in GHz
dB_noise = -114;           # noise power in dBm
dB_Pd_max = 23            # d2d maximum power
dB_Pc_max = 23              # CUE maximum power
noise = 10**(dB_noise/10)
Pd_max = 10**(dB_Pd_max/10)
Pc_max = 10**(dB_Pc_max/10)
numCUE = 200
numV2V = 10
coordCUE = np.zeros((numCUE,2))
########## User Initialization #################
x,y = genCUE(-500,25,-500,500,numCUE/2)
x1,y1 = genCUE(41,500,-500,500,numCUE/2)
for i in range(numCUE) :
  if i < numCUE/2 :
    coordCUE[i][0] = x[i]
    coordCUE[i][1] = y[i]
  else :
    coordCUE[i][0] = x1[i]
    coordCUE[i][1] = y1[i]

d2dPos,indDUE,indDUE2 = genDUE(numV2V)
indDUE = [int(i) for i in indDUE]
indDUE2 = [int(i) for i in indDUE2]

############### Resource Block allocation ####################
queue = []
for i in range(10000) :
  if i%10 == 0 :
    for V2Vuser in range(numV2V) :
      if len(queue) < 500 :
        dist = sqrt((d2dPos[indDUE[V2Vuser]][0][0] - d2dPos[indDUE2[V2Vuser]][0][0])**2 + (d2dPos[indDUE[V2Vuser]][0][1] - d2dPos[indDUE2[V2Vuser]][0][1])**2)
        PL = pathloss('V2V',fc,dist)
        PL = 10**(PL/10)
        x = [10,PL,'V2V']
        queue.append(x)

  if i%20== 0 :
    CUE_genpacket = []
  numCUE_genpacket = np.random.poisson(7.2) 
  while i <  numCUE_genpacket :
    CUE = randint(0,numCUE-1)
    if CUE in  CUE_genpacket :
      continue
    else :
      if len(queue) < 500 : 
       dist = sqrt(coordCUE[CUE][0]**2 + coordCUE[CUE][1]**2)
       PL = pathloss('CUE',fc,dist)
       PL = 10**(PL/10)
       x = [10,PL,'V2V']
       queue.append(x)



In [42]:
d2dPos,indDUE,indDUE2 = genDUE(10)
#print(d2dPos)
#print(indDUE)
#print(indDUE2)
indDUE = [int(i) for i in indDUE]
indDUE2 = [int(i) for i in indDUE2]
print(d2dPos[indDUE2[0]][0][1])



25.0
